
<h1><center><b>Project Two</b></center></h1>

<h3><center><i>Jordan Sinclair</i></center></h3>

# <h1><center><b>Notebook Setup</b></center></h1>

When you are done with this notebook, run the following code cell to unmount Google Drive

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


The following code cell installs additional dependencies required to run the Jupyter Notebooks used in this class.



In [ ]:
# Add additional Python packages that we will be using in class
!apt install graphviz build-essential checkinstall imagemagick
# Base Python packages to run example Jupyter Notebooks
!pip install watermark pyprind mlxtend
# Python packages to visualize Decision Tree Classifiers
!pip install pydotplus graphviz pyparsing
# Python packages for Natrual Language Processing
!pip install nltk
# Python packages for Flask-based web applications
!pip install flask wtforms
# Python packages for TensorFlow
!pip install tensorflow tensorflow-datasets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  file fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagic-mgc libmagic1 libmagickcore-6.q16-3
  libmagickcore-6.q16-3-extra libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7
  netpbm poppler-data
Suggested packages:
  gettext fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr
  | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray
  radiance sane-utils texlive-base-bin transfig ufraw-batc

The following code cell defines a variable equal to the location inside your Google Drive where you copied the ch03 folder

In [ ]:
##### TODO CHANGE THIS TO THE PATH IN GOOGLE DRIVE WHERE YOU COPIED THE ch03 FOLDER #####
google_drive_root='/machine-learning/Project Two Jordan Sinclair'

In [ ]:
google_drive_mount_location = '/content/drive'
google_file_prefix=google_drive_mount_location + '/My Drive/' + google_drive_root + '/'

The following code cell mounts your Google Drive into the runtime of the workbook, so that you can access files.

In [ ]:
# Read more here: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=D78AM1fFt2ty
from google.colab import drive
drive.mount(google_drive_mount_location)

Mounted at /content/drive


# <center><b>Imports</b></center>

In [ ]:
#Imports for data preprocessing:
import numpy as np
import pandas as pd
from io import StringIO
import sys
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.porter import PorterStemmer
import os

#For Website Creation:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import sqlite3

#For machine learning models:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

# <h1><center><b>Experiment Objective</b></center></h1>

I wanted to create a machine learning algorithm that will take a user's Ben & Jerry's icecream review and describe whether it is positive or negative. I originally planned to use multiclass categorization with positive, negative, and neutral options, but the neutral class didn't was very small in comparison to the other two and the model had trouble differentiating it. 

The data I chose to use came from a Kaggle dataset that contained numerous ice cream related information. The link to the complete dataset is:

https://www.kaggle.com/tysonpo/ice-cream-dataset

I wasn't sure about the ethicality of using Ben & Jerry's reviews from their website. I couldn't find any kind of Terms of Service that covered this, so I decided to reach out to the company. They actually wrote back to me and explained that:

"Reviews on the web are public so feel free to use them for your project! Thanks for asking :)"

I really appreciate the company for taking time to respond to me. The data from Ben and Jerry's reviews had several features that weren't necessary for the sentiment analysis I wanted to complete. I decided to start by removing those extra columns, as described below. Afterwords, the dataset only consisted of the actual review text and the number of stars given. I decided to designate the negative class as 1, 2, or 3 stars and the positive as 4 or 5 stars. I decided to use the first 7000 samples of the shuffled data for the training set. This left about 1000 samples for the test dataset. 

At the end of this project, I hope to have a working website that will provide the overall sentiment of a user's input review and that will readjust the sentiment if the first estimate is incorrect. 

# <h1><center><b>Data Collection</b></center></h1>

I chose to use icecream review data from Ben & Jerrys' website. This data was retreived from a dataset on Kaggle that contained numerous ice cream related information, including different brand reviews, flavors, etc. 

In [ ]:
#Location of the dataset in Google Drive 
data = google_file_prefix + 'reviews.csv'

#If using a version before 3.0, the dataset must be converted to unicode.
if (sys.version_info < (3, 0)):
    data = unicode(data)

#Reads the csv data into a Pandas DataFrame, df. 
df = pd.read_csv(data, header=None, encoding='utf-8')

#Shows the first five rows of the dataset. 
df.head()

,0,1,2,3,4,5,6,7
0,key,author,date,stars,title,helpful_yes,helpful_no,text
1,0_bj,Ilovebennjerry,2017-04-15,3,Not enough brownies!,10,3,"Super good, don't get me wrong. But I came for..."
2,0_bj,Sweettooth909,2020-01-05,5,I’m OBSESSED with this pint!,3,0,I decided to try it out although I’m not a hug...
3,0_bj,LaTanga71,2018-04-26,3,My favorite...More Caramel Please,5,2,My caramel core begins to disappear about half...
4,0_bj,chicago220,2018-01-14,5,Obsessed!!!,24,1,Why are people complaining about the blonde br...


# <h1><center><b>Data Processing</b></center></h1>

First, I removed the first row of the dataset and relabeled the columns since the titles were in the first row. 

In [ ]:
df.columns = ["key", "author", "date", "stars", "title", "helpful_yes", "helpful_no", "text"]
df = df.drop(axis=0, index=0)
df.head()

,key,author,date,stars,title,helpful_yes,helpful_no,text
1,0_bj,Ilovebennjerry,2017-04-15,3,Not enough brownies!,10,3,"Super good, don't get me wrong. But I came for..."
2,0_bj,Sweettooth909,2020-01-05,5,I’m OBSESSED with this pint!,3,0,I decided to try it out although I’m not a hug...
3,0_bj,LaTanga71,2018-04-26,3,My favorite...More Caramel Please,5,2,My caramel core begins to disappear about half...
4,0_bj,chicago220,2018-01-14,5,Obsessed!!!,24,1,Why are people complaining about the blonde br...
5,0_bj,Kassidyk,2020-07-24,1,Worst Ice Cream Ever!,1,5,This ice cream is worst ice cream I’ve ever ta...


Next, I removed some of the columns, since I don't need all of the extra information, including the author, date, key, title, or helpfulness. 

In [ ]:
df = df.drop(axis=1, labels=["key", "author", "date", "helpful_yes", "helpful_no", "title"])

In [ ]:
df.head()

,stars,text
1,3,"Super good, don't get me wrong. But I came for..."
2,5,I decided to try it out although I’m not a hug...
3,3,My caramel core begins to disappear about half...
4,5,Why are people complaining about the blonde br...
5,1,This ice cream is worst ice cream I’ve ever ta...


Now the data can be shuffled. 

In [ ]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df

,stars,text
4838,5,I bought this a week ago for the first time an...
163,5,My mom and I always get this one every time we...
972,5,Searched high and low for this flavor. Finally...
5167,5,My favorite mint ice cream on the market and I...
1727,5,First of all everyone should try ice cream fla...
...,...,...
4932,5,"I picked this up with the Red, White and blueb..."
3265,3,Ben and Jerry's is my splurging ice cream. Onc...
1654,5,This was a flavor that I went to if I really w...
2608,5,This flavor and chunky monkey are by far my fa...


I wanted to change the five star ratings to a classification of either positive or negative sentiment. To do this, I first started by mapping the string numbers into integers starting from 0. 

In [ ]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['stars']))}
class_mapping

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4}

In [ ]:
#Converts the current class labels
df['stars'] = df['stars'].map(class_mapping)

In [ ]:
df

,stars,text
4838,4,I bought this a week ago for the first time an...
163,4,My mom and I always get this one every time we...
972,4,Searched high and low for this flavor. Finally...
5167,4,My favorite mint ice cream on the market and I...
1727,4,First of all everyone should try ice cream fla...
...,...,...
4932,4,"I picked this up with the Red, White and blueb..."
3265,2,Ben and Jerry's is my splurging ice cream. Onc...
1654,4,This was a flavor that I went to if I really w...
2608,4,This flavor and chunky monkey are by far my fa...


Next, I used the integer mappings to loc the star column values as either 0, for negative, or 1, for positive. I decided to have the negative class range from 1 to 3 stars and the positive 4 or 5 stars. 

In [ ]:
df.loc[df["stars"] == 0, "stars"]=0
df.loc[df["stars"] == 1, "stars"]=0
df.loc[df["stars"]==2, "stars"]=0
df.loc[df["stars"] == 3, "stars"]=1
df.loc[df["stars"] == 4, "stars"]=1

In [ ]:
df

,stars,text
4838,1,I bought this a week ago for the first time an...
163,1,My mom and I always get this one every time we...
972,1,Searched high and low for this flavor. Finally...
5167,1,My favorite mint ice cream on the market and I...
1727,1,First of all everyone should try ice cream fla...
...,...,...
4932,1,"I picked this up with the Red, White and blueb..."
3265,0,Ben and Jerry's is my splurging ice cream. Onc...
1654,1,This was a flavor that I went to if I really w...
2608,1,This flavor and chunky monkey are by far my fa...


The data is saved into a new csv file below so that the row indeces will start at 0 again. 

In [ ]:
df.to_csv(google_file_prefix + 'bj_data.csv', index=False, encoding='utf-8',)

This results in the following dataset:

In [ ]:
df = pd.read_csv(google_file_prefix + 'bj_data.csv', encoding='utf-8')
df.head()

,stars,text
0,1,I bought this a week ago for the first time an...
1,1,My mom and I always get this one every time we...
2,1,Searched high and low for this flavor. Finally...
3,1,My favorite mint ice cream on the market and I...
4,1,First of all everyone should try ice cream fla...


Before the text data can be used in a machine learning model, it must be cleaned. The following code block defines a function the cleans the text by removing all of the unwanted characters and converting the text to lowercase.

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

Now this function can be applied to the text column in the data frame. 

In [ ]:
df['text'] = df['text'].apply(preprocessor)

In [ ]:
df

,stars,text
0,1,i bought this a week ago for the first time an...
1,1,my mom and i always get this one every time we...
2,1,searched high and low for this flavor finally ...
3,1,my favorite mint ice cream on the market and i...
4,1,first of all everyone should try ice cream fla...
...,...,...
7938,1,i picked this up with the red white and bluebe...
7939,0,ben and jerry s is my splurging ice cream once...
7940,1,this was a flavor that i went to if i really w...
7941,1,this flavor and chunky monkey are by far my fa...


The following cell defines and applies a bag of words model to the text in the dataframe by utilizing scikit learn's Count Vectorizer. 

In [ ]:
count = CountVectorizer()
bag = count.fit_transform(df.text)

This raw term frequencies from the Count Vectorizer are then used to construct a Term Frequency Inverse Document Frequency transformer that is implemented to downweight frequently occuring words. 

In [ ]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(df['text'])).toarray()
raw_tfidf 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Adjusts the TFIDF data using L2 normalization. 

In [ ]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

From the resulting arrays above, it was difficult to see whether they were completely filled with zeros or not. To ensure that they had some numerical values, I used numpy's any function, which returns true if there are any nonzero elements in the array. 

In [ ]:
np.any(l2_tfidf)

True

In [ ]:
np.any(raw_tfidf)

True

The following functions are used to tokenize the data. The Natural Language Tool Kit's function, PorterStemmer, is used to trim the end of words such as 'talking' to 'talk'. 

In [ ]:
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

Removes the stop words:

In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
for i in df["text"]:
  [w for w in tokenizer_porter(i) if w not in stop]

This cell splits the dataset into training and testing data. I decided to use a 30% test size. 

In [ ]:
X_train = df.loc[:5560, 'text'].values
y_train = df.loc[:5560, 'stars'].values
X_test = df.loc[5560:, 'text'].values
y_test = df.loc[5560:, 'stars'].values
print("Ratio of training to testing data: ", len(X_train), "/", len(X_test))

Ratio of training to testing data:  5561 / 2383


In [ ]:
print(X_train[1000])

if my husband and i want a snack on cheat day this is our at home go to its simple yet amazing and its hard to stop eating it 


In [ ]:
print(y_train[1000])

1


In [ ]:
print(X_train[1006])

not very flavorful the shortbread adds a gritty mushy texture i couldn t taste raspberry the blueberry was dull the strawberry just didn t work in the mix 


In [ ]:
print(y_train[1006])

0


# <center><b>Model Optimization and Serialization</b></center>

The following code defines a parameter grid for the grid search. I am using a SGD Classifier, the Count Vectorizer defined above, and the TFIDF transformer defined above in a pipeline. The grid search incorporates different options for each element in the pipeline. For the SGD Classifier, the alpha value, type of loss, and penalty type are varied. The use of stop words and the ngram range are changed for the Count Vectorizer. Finally, for the TFIDF transformer, the use of the 'idf' is varied. 

In [ ]:
param_grid = [{'vect__stop_words': [stop, None],
               'vect__ngram_range': ((1, 1), (1, 2)),
               'clf__alpha': [0.001, 0.005, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100],
               'clf__loss': ['log', 'hinge', 'perceptron'],
               'clf__penalty': ['l1', 'l2']},
              {'vect__ngram_range': ((1, 1), (1, 2)),
               'vect__stop_words': [stop, None],
               'tfidf__use_idf':[True, False],
               'tfidf__norm':[None,'l1', 'l2'],
               'clf__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 5, 10, 50, 100],
               'clf__loss': ['log', 'hinge', 'perceptron'],
               'clf__penalty': ['l1', 'l2']}]
sgd_tfidf = Pipeline([('vect', count),
                      ('tfidf', tfidf),
                      ('clf', SGDClassifier())])

gs_sgd_tfidf = GridSearchCV(sgd_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

The grid search is fitted to the training data in order to find the best hyperparameters. 

In [ ]:
gs_sgd_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 1680 candidates, totalling 8400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done 3277 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 34.1min
[Parallel(n_jobs=-1)]: Done 4897 tasks      | elapsed: 39.8min
[Parallel(n_jobs=-1)]: Done 5828 tasks      | elapsed: 45.9min
[Parallel(n_jobs=-1)]: Done 6841 tasks      | elapsed: 52.7min
[Parallel(n_jobs=-1)]: Done 7934 tasks      | elapsed: 59.8min
[Parallel(n_jobs=-1)]: Done 8400 out of 8400 |

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

The best parameters from the grid search are printed and used to give the CV accuracy for the pipeline. The best parameters for the pipeline are:



1.   For the SGD Classifier:


  *   Alpha: 0.1
  *   Loss: hinge
  *   Penalty: L2


2.   For the Count Vectorizer:

*   N Gram Range: (1,2)
*   Stop Words: none

3. For the TFIDF Vectorizer:

*   Norm: None
*   Use IDF: True



Additionally, the optimized pipeline performed very well in terms of accuracy, scoring 92.4%. 

In [ ]:
print('Best parameter set: %s ' % gs_sgd_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_sgd_tfidf.best_score_)

Best parameter set: {'clf__alpha': 0.1, 'clf__loss': 'hinge', 'clf__penalty': 'l2', 'tfidf__norm': None, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2), 'vect__stop_words': None} 
CV Accuracy: 0.924


The classifier is set with the grid search results and scored based on the unseen testing data. It also performed quite well with an accuracy of 92.1%

In [ ]:
clf = gs_sgd_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.921


The following block uses an out-of-core learning method to clean, process, stream, vectorize, and model the data. 

In [ ]:
stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
          if line[0].isnumeric() :
            text, label = line[2:], int(line[0])
            yield text, label
variable = stream_docs(path=google_file_prefix + 'bj_data.csv')
def get_minibatch(doc_stream, size):
  docs, y = [], []
  for _ in range(size):
    foo = next(doc_stream)
    text, label = foo[0], foo[1]
    docs.append(text)
    y.append(label)
  return docs, y

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path=google_file_prefix + 'bj_data.csv')
print(next(doc_stream))
import pyprind

classes = np.array([0, 1, 2])
X, y = get_minibatch(doc_stream, size=7942)
X_train, y_train = X[:5560], y[:5560]
X_test, y_test = X[5560:], y[5560:]
if not X_train:
  print('break')
X_train = vect.transform(X_train)
clf.partial_fit(X_train, y_train, classes=[0, 1, 2])

('I bought this a week ago for the first time and instantly fell in love. The combination is epic!\n', 1)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=1, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [ ]:
len(df)

7943

In [ ]:
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.902


In [ ]:
clf = clf.fit(X_test, y_test)
clf

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=1, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

## Serializing the Trained Model:

In [ ]:
import pickle
import os
from sklearn.feature_extraction.text import HashingVectorizer
import re

The following section creates a standalone Python file that is used in making a Flask application in the next section. It implements a hashing vectorizer using the optimized tfidf vectorizer. 

In [ ]:
#Gets the destination for the pickled objects. 
dest = os.path.join(google_file_prefix + 'creamyclassifier', 'pkl_objects')
#Creates the directory
if not os.path.exists(dest):
    os.makedirs(dest)
#Adds the pickled stop and classifier objects to the destination
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [ ]:
os.chdir(google_file_prefix)

Writes the vectorizer file that will conain the tokenizer and hashing vectorizer. 

In [ ]:
%%writefile creamyclassifier/vectorizer.py
#Gets the current directory of the file. 
cur_dir = os.path.dirname(__file__)
#Loads the stopword data pickled earlier. 
stop = pickle.load(open(os.path.join(
                cur_dir,
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))
#Tokenizer function that cleans the text data and removes the stop words. 
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized
#Hashing vectorizer that uses the tokenizer above. 
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting creamyclassifier/vectorizer.py


In [ ]:
os.chdir(google_file_prefix + 'creamyclassifier')

In [ ]:
#Imports the hashing vectorizer created above. 
from vectorizer import vect
#Loads the classifier that was pickled earlier. 
clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

Examples:

In [ ]:
label = {0:'negative', 1:'positive'}

example = ["I love this flavor. It's amazing."]
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 99.77%


In [ ]:
ex2 = ["I hate this icecream! It's horrible"]
X2 = vect.transform(ex2)
print("Prediction: %s\nProbability: %.2f%%" %\
      (label[clf.predict(X2)[0]],
       np.max(clf.predict_proba(X2))*100))

Prediction: negative
Probability: 58.97%


# <center><b>Website Creation and Publishing</b></center>

## SQLite Database:

In [ ]:
os.getcwd() #Gets the current working directory

'/content/drive/My Drive/machine-learning/Project Two Jordan Sinclair/creamyclassifier'

In [ ]:
#Connects to the database and gets the cursor. 
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
#Sets up the table for the review and sentiment data
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER)')
#Examples:
example1 = 'I love this flavor'
c.execute("INSERT INTO review_db (review, sentiment) VALUES (?, ?)", (example1, 1))

example2 = 'This flavor was not good'
c.execute("INSERT INTO review_db (review, sentiment) VALUES (?, ?)", (example2, 0))
#Closes the connection
conn.commit()
conn.close()

In [ ]:
#Connects to the database again and gets the cursor. 
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
#Gets all of the data input into the database. 
c.execute("SELECT * FROM review_db")
results = c.fetchall()
#Closes the connection
conn.close()

In [ ]:
#Prints the results from the database. 
print(results)

[('I love this flavor', 1), ('This flavor was not good', 0)]


## Flask Application:

The link to my website is:

http://artengine.pythonanywhere.com/

